In [ ]:
from astropy.io import fits
from astropy.utils.data import get_pkg_data_filename

import os
import glob

import numpy as np

In [ ]:
pathToPos = 'top1500processed/top1500processed_pos/'
pathToNeg = 'top1500processed/top1500processed_neg/'

In [ ]:
#pathToPos = 'PositiveWithDESSky/'
#pathToNeg = 'DES/DES_Processed'

In [ ]:
folders = {}
for root, dirs, files in os.walk(pathToPos):
    for folder in dirs:
        key = folder
        value = os.path.join(root, folder)
        folders[key] = value
        
# subf = []
# for folder in folders:
#     subf.append(folder[len(pathToPos)])

In [ ]:
#number of Positive DataPoints
nDT = len(folders)

DataPos = np.zeros([nDT, 3, 100, 100])

# key is name of folder number
# value is the number of the folder to be added to the file name

counter = 0
for key, value in folders.items():
    g_name = get_pkg_data_filename(value + '/' + str(key) + '_posSky_g_norm.fits')
    r_name = get_pkg_data_filename(value + '/' + str(key) + '_posSky_r_norm.fits')
    i_name = get_pkg_data_filename(value + '/' + str(key) + '_posSky_i_norm.fits')
    
#    g_name = get_pkg_data_filename(folders[var]+'/'+subf[var]+'g_WCSClipped.fits')
#    r_name = get_pkg_data_filename(folders[var]+'/'+subf[var]+'r_WCSClipped.fits')
#    i_name = get_pkg_data_filename(folders[var]+'/'+subf[var]+'i_WCSClipped.fits')
    
    
    g = fits.open(g_name)[0].data[0:100,0:100]
    r = fits.open(r_name)[0].data[0:100,0:100]
    i = fits.open(i_name)[0].data[0:100,0:100]
    
    DataPos[counter] = [g, r, i] 
    counter += 1
#    if counter > 1500:
#        break

In [ ]:
#Loading negative examples

# r=root, d=directories, f = files

foldersNeg = []
for root, dirs, files in os.walk(pathToNeg):
    for folder in dirs:
        foldersNeg.append(os.path.join(root, folder))

In [ ]:
nDT = len(foldersNeg)
DataNeg = np.zeros([nDT,3,100,100])

for var in range(len(folders)):
#    g_name = get_pkg_data_filename(foldersNeg[var]+'/g_WCSClipped.fits')
#    r_name = get_pkg_data_filename(foldersNeg[var]+'/r_WCSClipped.fits')
#    i_name = get_pkg_data_filename(foldersNeg[var]+'/i_WCSClipped.fits')    

    g_name = get_pkg_data_filename(foldersNeg[var]+'/g_norm.fits')
    r_name = get_pkg_data_filename(foldersNeg[var]+'/r_norm.fits')
    i_name = get_pkg_data_filename(foldersNeg[var]+'/i_norm.fits')    

    g = fits.open(g_name)[0].data[0:100,0:100]
    r = fits.open(r_name)[0].data[0:100,0:100]
    i = fits.open(i_name)[0].data[0:100,0:100]    
    
    DataNeg[var] = [g, r, i]
#    if var > 1500:
#        break

In [ ]:
def norm(x):
    m = x.mean()
    v = x.std()
    return (x-m)/v

In [ ]:
from matplotlib import pyplot
im2disp = DataNeg[20].transpose((1,2,0))
pyplot.imshow(im2disp)
pyplot.show()

In [ ]:
im2disp = DataPos[20].transpose((1,2,0))
pyplot.imshow(im2disp)
pyplot.show()

In [ ]:
DataPos.std()

I have found an issue here:
   DEF: The std is the quantity expressing by how much the members of a group differ from the mean value for the group.
   
   DataPos has a very low std, and this means that all the images are very similar. This is not exactly what we want as this doesnt have a huge difference when training, testing or validating. 
   
   Since DataNeg std is so much bigger, there is quite a large variety

In [ ]:
DataNeg.std()

In [ ]:
AllData = np.vstack(DataPos, DataNeg)

In [ ]:
AllData.std()

In [ ]:
# Gaussian normalization of the data
for i in range(DataPos.shape[0]):
    for j in range(DataPos.shape[1]):
        DataPos[i,j] = norm(DataPos[i,j])
        #print(DataPos.std())

for i in range(DataNeg.shape[0]):
    for j in range(DataNeg.shape[1]):
        DataNeg[i,j] = norm(DataNeg[i,j])
        #print(DataNeg.std())
        

In [ ]:
from matplotlib import pyplot
im2disp = DataNeg[20].transpose((1,2,0))
pyplot.imshow(im2disp)
pyplot.show()

In [ ]:
im2disp = DataPos[20].transpose((1,2,0))
pyplot.imshow(im2disp)
pyplot.show()

In [ ]:
# We need to create train and test "datasets",
# let's say 80% images for training and 20% for test from every group

In [ ]:
DataPos.shape[0]
DataNeg.shape[0]

In [ ]:
import numpy.random as rnd
rnd.seed(2019) #fix seed for reproducibility of results 

listPos = list(np.arange(DataPos.shape[0]))
listPosTest = list(rnd.choice(listPos,int(DataPos.shape[0]*0.2), replace=False))
listPosRem = list(set(listPos)-set(listPosTest))
listPosVal = list(rnd.choice(listPosRem,int(DataPos.shape[0]*0.2), replace=False))
listPosTrain = list(set(listPosRem)-set(listPosVal))


listNeg = list(np.arange(DataPos.shape[0],DataPos.shape[0]+DataNeg.shape[0]))
listNegTest  = list(rnd.choice(listNeg,int(DataNeg.shape[0]*0.2), replace=False))
listNegRem = list(set(listNeg)-set(listNegTest))
listNegVal  = list(rnd.choice(listNegRem,int(DataNeg.shape[0]*0.2), replace=False))
listNegTrain = list(set(listNegRem)-set(listNegVal))

listTest  = listPosTest  + listNegTest
rnd.shuffle(listTest)
listVal  = listPosVal  + listNegVal
rnd.shuffle(listVal)
listTrain = listPosTrain + listNegTrain
rnd.shuffle(listTrain)

In [ ]:
#Now we are ready to create X_train, Y_train and X_test and Y_test

Ntest  = len(listTest)
Nval   = len(listVal)
Ntrain = len(listTrain)

X_train = np.zeros([Ntrain,3,100,100])
Y_train = np.zeros(Ntrain, dtype=int)

X_test = np.zeros([Ntest,3,100,100])
Y_test = np.zeros(Ntest, dtype=int)

X_val = np.zeros([Nval,3,100,100])
Y_val = np.zeros(Nval, dtype=int)


for i in range(Ntest):
    if listTest[i]<DataPos.shape[0]:
        X_test[i] = DataPos[listTest[i]]
        Y_test[i] = 1
    else:
        X_test[i] = DataNeg[listTest[i]-DataPos.shape[0]]
        Y_test[i] = 0

for i in range(Nval):
    if listVal[i]<DataPos.shape[0]:
        X_val[i] = DataPos[listVal[i]]
        Y_val[i] = 1
    else:
        X_val[i] = DataNeg[listVal[i]-DataPos.shape[0]]
        Y_val[i] = 0        
        
for i in range(Ntrain):
    if listTrain[i]<DataPos.shape[0]:
        X_train[i] = DataPos[listTrain[i]]
        Y_train[i] = 1
    else:
        X_train[i] = DataNeg[listTrain[i]-DataPos.shape[0]]
        Y_train[i] = 0

In [ ]:
X_test  = X_test.transpose(0,2,3,1)
X_val   = X_val.transpose(0,2,3,1)
X_train = X_train.transpose(0,2,3,1)

In [ ]:
#Now we are almost ready to create CNN :)

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Flatten, Dense, Dropout

In [ ]:
model = Sequential()
model.add(Conv2D(8, kernel_size = (3, 3), activation='relu', input_shape=(100, 100, 3)))
model.add(MaxPooling2D(pool_size=(4,4)))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [ ]:
model.fit(X_train, Y_train, epochs=30, batch_size=200, validation_data=(X_val, Y_val))

In [ ]:
# You can add more layers, DropOut and others

In [ ]:
model.save_weights('model_baseline.h5')

In [ ]:
_, acc = model.evaluate(X_test, Y_test, verbose=0)
print("accuracy on the test set ->", acc * 100.0)

In [ ]:
np.round(model.predict(X_test))

In [ ]:
model.summary()

In [ ]:
DataPos.dump('DataPos.pkl')
DataNeg.dump('DataNeg.pkl')

# Test 84 good and 84 bad images.

In [ ]:
# get path names for good and bad sources, where good sources are the 
# known sources and the bad sources are the unknown sources

pathToKnown = 'KnownLenses/Known_Processed'
pathToUnknown = 'DES/Unknown'

In [ ]:
# Loading known examples from Jacobs paper

# r=root, d=directories, f = files

foldersKnown = []
for root, dirs, files in os.walk(pathToKnown):
    for folder in dirs:
        foldersKnown.append(os.path.join(root, folder))

In [ ]:
nDT = len(foldersKnown)
DataKnown = np.zeros([nDT,3,100,100])

for var in range(len(foldersKnown)):
#    g_name = get_pkg_data_filename(foldersKnown[var]+'/g_WCSClipped.fits')
#    r_name = get_pkg_data_filename(foldersKnown[var]+'/r_WCSClipped.fits')
#    i_name = get_pkg_data_filename(foldersKnown[var]+'/i_WCSClipped.fits')    

    g_name = get_pkg_data_filename(foldersKnown[var]+'/g_norm.fits')
    r_name = get_pkg_data_filename(foldersKnown[var]+'/r_norm.fits')
    i_name = get_pkg_data_filename(foldersKnown[var]+'/i_norm.fits')    

    g = fits.open(g_name)[0].data[0:100,0:100]
    r = fits.open(r_name)[0].data[0:100,0:100]
    i = fits.open(i_name)[0].data[0:100,0:100]    
    
    DataKnown[var] = [g, r, i]

In [ ]:
# Loading unknown examples from DES

foldersUnknown = []
for root, dirs, files in os.walk(pathToUnknown):
    for folder in dirs:
        foldersUnknown.append(os.path.join(root, folder))

In [ ]:
nDT = len(foldersUnknown)
DataUnknown = np.zeros([nDT,3,100,100])

for var in range(len(foldersUnknown)):
#    g_name = get_pkg_data_filename(foldersUnknown[var]+'/g_WCSClipped.fits')
#    r_name = get_pkg_data_filename(foldersUnknown[var]+'/r_WCSClipped.fits')
#    i_name = get_pkg_data_filename(foldersUnknown[var]+'/i_WCSClipped.fits')    

    g_name = get_pkg_data_filename(foldersUnknown[var]+'/g_norm.fits')
    r_name = get_pkg_data_filename(foldersUnknown[var]+'/r_norm.fits')
    i_name = get_pkg_data_filename(foldersUnknown[var]+'/i_norm.fits')    

    g = fits.open(g_name)[0].data[0:100,0:100]
    r = fits.open(r_name)[0].data[0:100,0:100]
    i = fits.open(i_name)[0].data[0:100,0:100]    
    
    DataUnknown[var] = [g, r, i]

In [ ]:
DataKnown = DataKnown.transpose(0, 3, 2, 1)
np.round(model.predict(DataKnown))
print ("Non zeros: " +str(np.count_nonzero(model.predict(DataKnown))))
print ("length: "+ str(len(DataKnown)))

In [ ]:
DataUnknown = DataUnknown.transpose(0, 3, 2, 1 )
np.round(model.predict(DataUnknown))
print ("Non zeros: " +str(np.count_nonzero(model.predict(DataUnknown))))
print ("length: "+ str(len(DataUnknown)))